## Performing 2-step registration of Sparse-dense point clouds

In [131]:
import open3d as o3d
import copy
import numpy as np
import time

Use downsampled lidar for faster computation

In [132]:
lidar=o3d.io.read_point_cloud("Data/lidar.txt", format='xyz')
downsampled_lidar=lidar.voxel_down_sample(0.02)
trajectory = o3d.io.read_point_cloud("Data/test_traj.txt", format='xyz')
axes=o3d.io.read_point_cloud("Data/coord_axes.txt", format='xyzrgb')
trajectory.paint_uniform_color([0, 0, 0])

PointCloud with 17523 points.

In [133]:
intervals = o3d.io.read_point_cloud("Data/intervals.txt", format='xyz')
intervals.paint_uniform_color([1, 0, 0])

rotation_vector=np.loadtxt("Data/rotations.txt")


def spc_reader(image_num):
    '''
    Reads sparse-point-clouds from the directory
    '''
    iteration_number=image_num
    imageNo=str(1583+iteration_number)
    spc = o3d.io.read_point_cloud("Data/spc_5m/DSC0"+imageNo+".txt", format='xyz')
    
    return spc

In [134]:
def translationVector(image_num):
    '''
    adding bias based on data
    '''
    translation_vector=np.array(intervals.points)
    tr=[]
    tr.append(translation_vector[image_num,0]+1.5)
    tr.append(translation_vector[image_num,1]+3.4)
    tr.append(translation_vector[image_num,2])
    #print("translation:",tr)
    return tr    

In [135]:
def rotationVector(spc,image_num):
    '''
    initial rotations based on data
    '''
    rotation=rotation_vector[image_num].tolist()
    R = spc.get_rotation_matrix_from_xyz(rotation)
    return R

In [136]:
def initial_transform(spc,T,R):
    '''
    based on initial rotations and tranlations
    '''
    spcT = copy.deepcopy(spc)
    spcT.rotate(R)
    spcT = spcT.translate(tr)
    #spcT.paint_uniform_color([1,0,1])
    
    return spcT

In [120]:
#o3d.visualization.draw_geometries([intervals,axes, spcT, downsampled_lidar])

Function to visualize registration result

In [137]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    s_c=copy.deepcopy(source)
    s_c.paint_uniform_color((0,0,1))
    #FOR FLIPPING POINT CLOUD UPSIDE DOWN
    #source_temp.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    #target.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    o3d.visualization.draw_geometries([source_temp, target, axes],
                                      zoom=0.1,
                                      front=[-0.2458, -0.8088, 0.5342],
                                      lookat=[0, 0, 0],
                                      up=[0.3109, -0.5878, -0.7468])

### Point-to-plane ICP registration

In [138]:
def PointToPlaneICP(spcT, dL):
    '''
    initially transformed sparse point-cloud: spcT is aligned with down-sampled lidar: dL for estimating
    final transformation parameters
    '''
    
    source=spcT
    #target=o3d.io.read_point_cloud("Data/down_small_lidar.txt", format='xyz')
    target=copy.deepcopy(dL)
    current_transformation = np.identity(4)

    #print("2. Point-to-plane ICP registration is applied on original point")
    #print("   clouds to refine the alignment. Distance threshold 2.")

    source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    #start = time.time()
    result_icp = o3d.pipelines.registration.registration_icp(
        source, target, 2, current_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    #print("Point-to-Plane ICP registration took %.3f sec.\n" % (time.time() - start))
    #print(result_icp)
    #print(result_icp.transformation)
    inlier_rmse=np.array(result_icp.inlier_rmse)
    correspondence_set_size=np.array(len(result_icp.correspondence_set))
    transformations=np.array(result_icp.transformation)
    
    return inlier_rmse, correspondence_set_size, transformations

#draw_registration_result_original_color(source, target, result_icp.transformation)

In [139]:
def registrationTwoStep(image_num, d_L):
    '''
    function for complete pipeline of registration
    '''
    
    i=image_num
    sparse_point_cloud=spc_reader(i)
    T=translationVector(i)
    R=rotationVector(sparse_point_cloud,i)
    spcT=initial_transform(sparse_point_cloud,T,R)
    RMSE, Size, Transforms = PointToPlaneICP(spcT, d_L)
    
    return RMSE, Size, Transforms

In [140]:
result_folder="Data/Results"
dataset_size=74

inlier_rmse=[]
correspondence_set_size=[]
transformations=[]

start = time.time()
for i in range(dataset_size):
    rmse, size, transforms=registrationTwoStep(i, downsampled_lidar)
    inlier_rmse.append(rmse)
    correspondence_set_size.append(size)
    transformations.append(transforms)
    
print("Complete Batch registration took %.3f sec.\n" % (time.time() - start))

Complete Batch registration took 4868.820 sec.



In [141]:
inlier_rmse

[array(0.09595444),
 array(0.09568977),
 array(0.09475652),
 array(0.0970854),
 array(0.10139227),
 array(0.11636438),
 array(0.12956322),
 array(0.14016871),
 array(0.15322942),
 array(0.11639661),
 array(0.12219212),
 array(0.1005733),
 array(0.27581351),
 array(0.26493132),
 array(0.22104838),
 array(0.2901721),
 array(0.2497545),
 array(0.2654301),
 array(0.34411857),
 array(0.31421292),
 array(0.37877076),
 array(0.34237201),
 array(0.43610056),
 array(0.3382053),
 array(0.28399585),
 array(0.32700537),
 array(0.10264681),
 array(0.12284093),
 array(0.16172802),
 array(0.24219076),
 array(0.3542235),
 array(0.32520818),
 array(0.33058666),
 array(0.3146887),
 array(0.32374997),
 array(0.34870576),
 array(0.36581934),
 array(0.37231807),
 array(0.30995004),
 array(0.38152669),
 array(0.43417415),
 array(0.41213139),
 array(0.41100679),
 array(0.38942466),
 array(0.37728135),
 array(0.41289243),
 array(0.41907963),
 array(0.20681287),
 array(0.26315237),
 array(0.22078972),
 array(0